In [1]:
#Subir la API key de Kaggle
from google.colab import files

print("Se necesita subir el archivo kaggle.json:")
files.upload()

Sube tu archivo kaggle.json:


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tgavh777","key":"5891ba7d79054ca84828d652f5e9981f"}'}

In [2]:
#Instalar kaggle, descargar y descomprimir el dataset
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

#Descarga el dataset
!kaggle datasets download -d vishesh1412/celebrity-face-image-dataset

# 3. Descomprime el .zip en la carpeta 'dataset_famosos'
!unzip -o -q celebrity-face-image-dataset.zip -d ./dataset_famosos/

Dataset URL: https://www.kaggle.com/datasets/vishesh1412/celebrity-face-image-dataset
License(s): CC0-1.0
  0% 0.00/52.9M [00:00<?, ?B/s]
100% 52.9M/52.9M [00:00<00:00, 1.38GB/s]
Descomprimiendo dataset...
¡Dataset listo!


In [3]:
#Borrado de famosos que no necesitamos
#Subimos nuestras fotos manualmente a la carpeta que se creo del dataset
!rm -rf "dataset_famosos/Celebrity Faces Dataset/Denzel Washington"
!rm -rf "dataset_famosos/Celebrity Faces Dataset/Johnny Depp"
!rm -rf "dataset_famosos/Celebrity Faces Dataset/Kate Winslet"
!rm -rf "dataset_famosos/Celebrity Faces Dataset/Robert Downey Jr"
!rm -rf "dataset_famosos/Celebrity Faces Dataset/Sandra Bullock"
!rm -rf "dataset_famosos/Celebrity Faces Dataset/Tom Cruise"
!rm -rf "dataset_famosos/Celebrity Faces Dataset/Tom Hanks"

In [4]:
#Sistema de reconocimiento facial

#Importaciones necesarias
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import os

#CONFIGURACIÓN Y PARÁMETROS
TAMAÑO_IMG = (224, 224)
TAMAÑO_LOTE = 32
EPOCAS_INICIALES = 20
EPOCAS_FINETUNING = 20

# Ruta del dataset
directorio_datos = './dataset_famosos/Celebrity Faces Dataset/'

# DATA AUGMENTATION
datagen_entrenamiento = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# GENERADORES DE ENTRENAMIENTO Y VALIDACIÓN
generador_entrenamiento = datagen_entrenamiento.flow_from_directory(
    directorio_datos,
    target_size=TAMAÑO_IMG,
    batch_size=TAMAÑO_LOTE,
    class_mode='categorical',
    subset='training'
)

generador_validacion = datagen_entrenamiento.flow_from_directory(
    directorio_datos,
    target_size=TAMAÑO_IMG,
    batch_size=TAMAÑO_LOTE,
    class_mode='categorical',
    subset='validation'
)

# Imprimir clases para verificar
print("\n--- CLASES DETECTADAS ---")
print(generador_entrenamiento.class_indices)
NUM_CLASES = generador_entrenamiento.num_classes

# 3. MODELO BASE (TRANSFER LEARNING)
modelo_base = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(TAMAÑO_IMG[0], TAMAÑO_IMG[1], 3)
)

#Bloqueo de los parámetros matemáticos de MobileNetV2.
modelo_base.trainable = False

# 4. ARQUITECTURA FINAL DEL MODELO
x = modelo_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predicciones = Dense(NUM_CLASES, activation='softmax')(x)

modelo = Model(inputs=modelo_base.input, outputs=predicciones)

# COMPILACIÓN FASE 1
modelo.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- INICIANDO ENTRENAMIENTO FASE 1 (BASE) ---")
mis_callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
]

historial = modelo.fit(
    generador_entrenamiento,
    epochs=EPOCAS_INICIALES,
    validation_data=generador_validacion,
    callbacks=mis_callbacks
)

# FINE-TUNING
modelo_base.trainable = True

inicio_ajuste_fino = len(modelo_base.layers) - 50
for capa in modelo_base.layers[:inicio_ajuste_fino]:
    capa.trainable = False

modelo.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

historial_ajuste = modelo.fit(
    generador_entrenamiento,
    epochs=EPOCAS_FINETUNING,
    validation_data=generador_validacion,
    callbacks=mis_callbacks
)

# GUARDAR MODELO FINAL
nombre_archivo = "modelo_facial_final.h5"
modelo.save(nombre_archivo)

Found 880 images belonging to 10 classes.
Found 220 images belonging to 10 classes.

--- CLASES DETECTADAS ---
{'Angelina Jolie': 0, 'Brad Pitt': 1, 'Hugh Jackman': 2, 'Jennifer Lawrence': 3, 'Leonardo DiCaprio': 4, 'Megan Fox': 5, 'Natalie Portman': 6, 'Nicole Kidman': 7, 'Scarlett Johansson': 8, 'Will Smith': 9}
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

--- INICIANDO ENTRENAMIENTO FASE 1 (BASE) ---


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - accuracy: 0.1903 - loss: 2.2901 - val_accuracy: 0.4045 - val_loss: 1.6170 - learning_rate: 0.0010
Epoch 2/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.3484 - loss: 1.7761 - val_accuracy: 0.4500 - val_loss: 1.4945 - learning_rate: 0.0010
Epoch 3/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - accuracy: 0.3922 - loss: 1.5671 - val_accuracy: 0.4955 - val_loss: 1.3256 - learning_rate: 0.0010
Epoch 4/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - accuracy: 0.4143 - loss: 1.5066 - val_accuracy: 0.5091 - val_loss: 1.3209 - learning_rate: 0.0010
Epoch 5/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.4213 - loss: 1.4554 - val_accuracy: 0.5500 - val_loss: 1.1600 - learning_rate: 0.0010
Epoch 6/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - accuracy: 0.5050 - loss: 1.3415 - val_accuracy: 0.6091 - val_loss: 1.0945 - learning_rate: 0.0010
Epoch 7/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.4968 - loss: 1.2585 - val_accuracy: 


¡Entrenamiento finalizado! Modelo guardado como: modelo_facial_final.h5
Descarga este archivo para usarlo en tu script de tiempo real.
